In [1]:
import bayesflow as bf
import keras
import numpy as np

Defaulting to JAX.
To override, set the KERAS_BACKEND environment variable before importing bayesflow.
See: https://keras.io/getting_started/#configuring-your-backend
/home/lars/Documents/code/python/bayesflow/.venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
INFO:2026-01-26 17:09:28,311:jax._src.xla_bridge:810: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
INFO:bayesflow:Using backend 'jax'


In [2]:
def prior():
    mu1 = -1 + np.random.standard_normal()
    mu2 = 1 + np.random.standard_normal()
    return {"mu": np.stack([mu1, mu2])}

In [3]:
def likelihood(mu, num_obs=10):
    return {"x": mu + np.random.standard_normal((num_obs, 2))}

In [4]:
simulator = bf.make_simulator([prior, likelihood])

In [5]:
joint_data = simulator.sample((32,))
contrastive_data = simulator.sample((32,))
contrastive_data["mu"] = simulator.sample((32,))["mu"]

In [6]:
nre_a = bf.RatioApproximator.nre_a(bf.networks.DeepSet(), bf.networks.MLP())

In [7]:
# nre_b = bf.RatioApproximator.nre_b(bf.networks.DeepSet(), bf.networks.MLP(), K=5)

In [8]:
nre_c = bf.RatioApproximator.nre_c(bf.networks.DeepSet(), bf.networks.MLP(), gamma=5, K=5)

In [9]:
nre_a.compile(optimizer="adam")
# nre_b.compile(optimizer="adam")
nre_c.compile(optimizer="adam")

In [10]:
nre_a.fit(epochs=10, simulator=simulator, num_batches=1000, batch_size=32, parameter_names=["mu"], observables_names=["x"])

INFO:bayesflow:Building dataset from simulator instance of SequentialSimulator.
INFO:bayesflow:Using 16 data loading workers.
INFO:bayesflow:Building on a test batch.


Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.4219
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2893
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2757
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2685
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2634
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2638
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2627
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2580
Epoch 9/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2636
Epoch 10/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2603


In [11]:
joint_ratio = nre_a.log_ratio(joint_data["mu"], joint_data["x"])
contrastive_ratio = nre_a.log_ratio(contrastive_data["mu"], contrastive_data["x"])

keras.ops.mean(joint_ratio, axis=0), keras.ops.mean(contrastive_ratio, axis=0)

(Array(2.1015859, dtype=float32), Array(-8.81812, dtype=float32))

In [12]:
# nre_b.fit(epochs=10, simulator=simulator, num_batches=1000, batch_size=32, parameter_names=["mu"], observables_names=["x"])

In [13]:
# joint_ratio = nre_b.log_ratio(joint_data["mu"], joint_data["x"])
# contrastive_ratio = nre_b.log_ratio(contrastive_data["mu"], contrastive_data["x"])
#
# keras.ops.mean(joint_ratio, axis=0), keras.ops.mean(contrastive_ratio, axis=0)

In [14]:
nre_c.fit(epochs=10, simulator=simulator, num_batches=1000, batch_size=32, parameter_names=["mu"], observables_names=["x"])

INFO:bayesflow:Building dataset from simulator instance of SequentialSimulator.
INFO:bayesflow:Using 16 data loading workers.
INFO:bayesflow:Building on a test batch.


Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.3166
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1802
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1675
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1654
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1614
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1571
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1570
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1556
Epoch 9/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1549
Epoch 10/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1540


In [15]:
joint_ratio = nre_c.log_ratio(joint_data["mu"], joint_data["x"])
contrastive_ratio = nre_c.log_ratio(contrastive_data["mu"], contrastive_data["x"])

keras.ops.mean(joint_ratio, axis=0), keras.ops.mean(contrastive_ratio, axis=0)

(Array(3.6851847, dtype=float32), Array(-8.825312, dtype=float32))